# <font color='black'> НИС: регрессионный анализ, 2025 </font>
# <font color='black'>Практическое занятие 1 </font>


## <font color='black'>Модель множественной линейной регрессии. Модели с переменными взаимодействия </font>
На этом занятии мы продолжим работать с данными из статьи [Kalenborn C., Lessman C., 2013](https://yadi.sk/i/nlEQUoWKiqY0UA). Одна из частей анализа в данной статье выполнена на основе cross-section data (использованы усредненные данные за 2005 - 2010 гг.). Авторы изучают взаимосвязь уровня коррупции и демократии, предполагая, что ее характер зависит от значений показателя свободы прессы. Кратко о данных:
* cpi - уровень коррупции: Corruption Perception Index. Непрерывная шкала от 0 до 10, где 10 означает наиболее высокий уровень коррупции.
* dem - индекс демократии: Vanhanen’s democratization index. Непрерывная шкала от 0 до 100, где 100 означает максимальное значение уровня демократии.
* fp - свобода прессы: Freedom House. Приведен к непрерывной шкале от 0 до 100, где 100 - наиболее высокое значение свободы прессы.
* loggdppc - натуральный логарифм ВВП на душу населения. World Bank.
* stab - уровень политической стабильности. Индекс построен на основе показателей "Political Stability" и "Absence of Violence/Terrorism" из the Worldwide Governance Indicators. Непрерывная шкала от -2.5 до 2.5, где 2.5 соответствует наиболее высокому уровню политической стабильности.
* britcol - дамми-переменная, где 1 - бывшая британская колония.

In [ ]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
from scipy.stats import norm, t

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

sns.set(style = "white", palette='deep')

Откроем массив данных для репликации результатов исследования - lab1.dta.

In [ ]:
lab1 = pd.read_stata('lab1.dta')
lab1 = lab1.dropna()

Познакомимся с тем, как устроен массив данных.

In [ ]:
lab1.head(10)

Для начала оценим парную линейную регрессию cpi на dem.

In [ ]:
m0 = smf.ols(formula = "cpi ~ dem", data = lab1).fit(cov_type = "HC3")
print(m0.summary())

Построим диаграмму рассеяния между зависимой переменной (cpi) и ключевым предиктором - уровнем демократии (dem)

In [ ]:
plt.figure(figsize=(8, 4))

scatter = sns.regplot(data=lab1, x="dem", y="cpi")

plt.title("Взаимосвязь индексов демократии и коррупции",
          fontsize=16, fontweight='bold')
plt.xlabel("Индекс демократии", fontsize=12, fontweight='bold')
plt.ylabel("Индекс восприятия коррупции", fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

Отдельно можно вывести вектор оценок параметров модели. Если хотите вывести оценку определенного параметра, то в квадратных скобках укажите его номер, не забываем, что в Python отсчет идет с 0, поэтому константе соответствует 0, а не 1.

In [ ]:
m0.params.iloc[1]

Также можно вывести ковариационную матрицу оценок параметров. Какая информация в ней содержится? По главной диагонали - дисперсии оценок параметров, остальные элементы - ковариации между оценками коэффициентов (совместная изменчивость). Если взять квадратный корень элементов по главной диагонали, то получим стандартные ошибки оценок коэффициентов, которые нам пригодятся при проверке гипотезы о незначимости коэффициентов в регрессионной модели.    

In [ ]:
m0.cov_params()

Представим полученный объект как матрицу в Python, чтобы можно было обратиться к определенным элементам. И давайте посмотрим, как рассчитать p-value в случае проверки гипотезы о незначимости коэффициента при предикторе dem против двусторонней альтернативы.

Статистика критерия рассчитывается следующим образом: $\dfrac{\hat{b}}{se}$. При верной нулевой гипотезе статистика имеет распределение Стьюдента с количеством степеней свободы равным $n-k-1$, где $k$ - количество предикторов в модели, не забываем про еще один параметр - это константа.

Сделайте вывод на основе полученного p-value.

In [ ]:
Cov_m0 = np.asmatrix(m0.cov_params())
t.sf(abs(m0.params.iloc[1]/np.sqrt(Cov_m0[1,1])), m0.nobs - m0.df_model - 1)*2

Для того же коэффициента построим 95%-ый доверительный интервал. В качестве отправной точки используем оценку соответствующего коэффициента $\hat{\beta}$ при dem. Также для того, чтобы задать границы, нам понадобится стандартная ошибка оценки коэффициента и критическая точка (квантиль по распределению Стьюдента уровня 0.975, $df = n - k - 1$).

Проинтерпретируйте полученный доверительный интервал.

In [ ]:
left_boundary = m0.params.iloc[1] - np.sqrt(Cov_m0[1,1])*t.ppf(0.975, m0.nobs - m0.df_model - 1)
right_boundary = m0.params.iloc[1] + np.sqrt(Cov_m0[1,1])*t.ppf(0.975, m0.nobs - m0.df_model - 1)

print(left_boundary.round(4), right_boundary.round(4))

И соответствующая таблица разложения вариации:

In [ ]:
anova = sm.stats.anova_lm(m0)
print(anova)

Оценим для начала регрессионную модель без переменных взаимодействия. Проинтерпретируйте все оценки коэффициентов модели m1. Как изменилась оценка коэффициента при dem по сравнению с соответствующей оценкой в модели m0?

In [ ]:
m1 = smf.ols(formula = "cpi ~ dem + fp + loggdppc + britcol + stab", data = lab1).fit(cov_type = "HC3")
print(m1.summary())

Альтернатива: Мы можем получить оценку коэффициента при любом предикторе в такой модели, воспользуясь теоремой Фриша-Во-Ловелла (the Frisch-Waugh-Lovell theorem). К примеру, для того, чтобы получить оценку коэффициента при переменной dem, а именно, оценить, как связаны dem и cpi, нужно очистить вариацию этих переменных от других предикторов (а именно, от fp, loggdppc, britcol и stab, с другой).

1) Для этого мы оцениваем регрессию dem на fp, loggdppc, britcol и stab, сохраняем остатки - то есть, получаем очищенный показатель dem.

2) Далее аналогично оцениваем регрессию cpi на fp, loggdppc, britcol и stab, сохраняем остатки - то есть, получаем очищенный показатель cpi.

3) После этого достаточно будет оценить регрессию очищенного cpi на очищенный dem и убедиться, что мы получили тот же самый коэффициент при dem, что и в исходной модели с контрольными переменными.

In [ ]:
m1_1 = smf.ols(formula = "dem ~ fp + loggdppc + britcol + stab", data = lab1).fit()
resid_data = pd.DataFrame()
resid_data["res1"] = m1_1.resid

In [ ]:
m1_2 = smf.ols(formula = "cpi ~ fp + loggdppc + britcol + stab", data = lab1).fit()
resid_data["res2"] = m1_2.resid

In [ ]:
m1_3 = smf.ols(formula = "res2 ~ res1", data = resid_data).fit(cov_type = "HC3")
print(m1_3.summary())

Рассмотрим, как различается уровень коррупции в группах стран с разным уровнем свободы прессы. Предварительно разделим страны на 2 группы: "not free" и "free". В качестве порогового значения для разделения возьмем 70 баллов по показателю "свобода прессы".

In [ ]:
lab1['groups'] = lab1['fp'].apply(lambda x: 1 if x > 70 else 0)

Модель 2 включает вместо непрерывной переменной уровня свободы прессы бинарный показатель "groups" в качестве предиктора. Проинтерпретируйте оценки коэффициентов модели m2.

In [ ]:
m2 = smf.ols(formula = "cpi ~ dem + groups + loggdppc + britcol + stab", data = lab1).fit(cov_type = "HC3")
print(m2.summary())

Тем не менее, данная модель не позволяет ответить на ключевой вопрос, которые ставили перед собой Kalenborn C., Lessman C. А именно, их интересовало, различается ли взаимосвязь коррупции и уровня демократии в странах с разными значениями свободы прессы? Для того, чтобы продвинуться в ответе на данный вопрос, представим следующий график. Какой предварительный вывод Вы можете сделать на основе данного графика?  

In [ ]:
fig = sns.lmplot(data=lab1, x="dem", y="cpi", hue="groups", legend=False)

plt.title("Взаимосвязь индексов демократии и коррупции", fontweight='bold')
plt.xlabel("Индекс демократии", fontweight='bold')
plt.ylabel("Индекс восприятия коррупции", fontweight='bold')

palette = sns.color_palette('deep', 2)
legend_elements = [
    mlines.Line2D([], [], color=palette[0], marker='o',
                 markersize=8, label='Несвободные'),
    mlines.Line2D([], [], color=palette[1], marker='o',
                 markersize=8, label='Свободные')
]

plt.legend(handles=legend_elements, loc='upper center',
           bbox_to_anchor=(0.5, -0.15), ncol=2, frameon=True)

plt.tight_layout()
plt.show()

Изменим спецификацию регрессионной модели так, чтобы она позволяла ответить на вопрос, как различается взаимосвязь коррупции и уровня демократии в двух группах стран (условно назовем их "свободные" и "несвободные"). Для этого нужно ввести переменную взаимодействия между предиктором "dem" и "groups".
Проинтерпретируйте оценки коэффициентов в модели m3.

In [ ]:
m3 = smf.ols(formula = "cpi ~ dem*groups + loggdppc + britcol + stab", data = lab1).fit(cov_type = "HC3")
print(m3.summary())

Проиллюстрируем на графике различия между предельными эффектами демократии в зависимости от групп стран. Для этого нам предварительно понадобится
* задать формулу для расчета предельных эффектов. Для этого мы

1) извлечем оценки коэффициентов модели m3 (marg_effect)

2) получим вектор уникальных значений переменной "groups" (values), которая является условием (то есть, от ее значений зависит предельный эффект демократии)

* задать формулу для расчета стандартных ошибок

In [ ]:
print(m3.params)
print(m3.cov_params())

In [ ]:
v = lab1.groups.unique()
me = m3.params.iloc[1] + m3.params.iloc[3] * v
cov = m3.cov_params().values
se = np.sqrt(cov[1,1] + v**2 * cov[3,3] + 2 * v * cov[1,3])

plt.figure(figsize=(7, 4))
plt.errorbar(me, v, xerr=1.96 * se, fmt='D', capsize=5)

plt.yticks([0, 1], ['Несвободные \n страны', 'Свободные \n страны'],
           fontsize=8, fontweight='bold')

for i, (y_val, x_val) in enumerate(zip(v, me)):
    plt.annotate(f'{x_val:.3f}', (x_val, y_val), xytext=(-15, -15),
                textcoords='offset points', va='center',
                fontsize=8.5, fontweight='bold',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.3, edgecolor='gray'))

plt.xlabel('Предельные эффекты', fontsize=10, fontweight='bold')

plt.grid(True, alpha=0.2)
plt.tight_layout()
plt.show()

Теперь попробуем оценить модель, но уже с переменной взаимодействия между исходными непрерывными переменными "dem" и "fp", без деления стран на группы "not free" и "free". Какой вывод можно сделать на основе оценок данной модели?
Предложите возможные преобразования для более удобной интерпретации и примените их при оценивании модели (в качестве дополнительной практики после занятия).

In [ ]:
m4 = smf.ols(formula = "cpi ~ dem*fp + loggdppc + britcol + stab", data = lab1).fit(cov_type = "HC3")
print(m4.summary())

Покажем на графике, как изменяется предельный эффект демократии и его значимость в зависимости от "условия" (freedom of press). Проинтерпретируйте график.

In [ ]:
cov_m4 = np.asmatrix(m4.cov_params())
fp_values = np.linspace(lab1.fp.min(), lab1.fp.max())
marg_effect_m4 = m4.params.iloc[1] + m4.params.iloc[3]*fp_values
se2 = np.sqrt(cov_m4[1,1] + fp_values**2*(cov_m4[3,3]) + 2*fp_values*cov_m4[1,3])

plt.figure(figsize=(7, 4))
plt.plot(fp_values, marg_effect_m4, 'b')
plt.plot(fp_values, marg_effect_m4 - norm.ppf(0.975)*se2, 'b--')
plt.plot(fp_values, marg_effect_m4 + norm.ppf(0.975)*se2, 'b--')
plt.xlabel('Свобода прессы', fontsize=10)
plt.ylabel('Предельный эффект демократии', fontsize=10)
plt.axhline(y=0, color ='r', linestyle = '--')
plt.title('Зависимость предельного эффекта \n от значений индекса свободы прессы', fontsize=12, fontweight='bold')

plt.grid(True, alpha=0.2)
plt.tight_layout()
plt.show()

Резюмируйте результаты проведенного анализа.